In [1]:
import numpy as np

from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format('bio_embedding_intrinsic', binary=True)

In [2]:
# trying to see how I might filter for drug names
# especially those with multi-word names
print(type(wv.vocab))
print('butabarbital-sodium' in wv.vocab.keys())
print('butabarbital' in wv.vocab.keys())

<class 'dict'>
False
True


In [3]:
# read in all the potential drug names and active ingredients
# dataset from: https://www.fda.gov/drugs/drug-approvals-and-databases/drugsfda-data-files
drug_names = set()
with open('fda_approved/Products.txt') as infile:
    header = infile.readline().strip().split('\t')
    name_ind = header.index('DrugName')
    actv_ing_ind = header.index('ActiveIngredient')
    # now go through and collect them all
    # TODO handle ; separated and parenthetical lists
    # TODO e.g. "drug;drug;drug;" and "drug (alias;alias;alias)"
    for l in infile:
        parts = l.strip().lower().split('\t')
        drug_names.add(parts[name_ind])
        drug_names.add(parts[actv_ing_ind])
# sort them into a list
drug_names = sorted(drug_names)
print(len(drug_names))
print('\n'.join(drug_names[:50]))

9019
8-hour bayer
8-mop
a-hydrocort
a-methapred
a-poxide
a.p.l.
a/t/s
abacavir and lamivudine
abacavir sulfate
abacavir sulfate and lamivudine
abacavir sulfate, lamivudine and zidovudine
abacavir sulfate; dolutegravir sodium; lamivudine
abacavir sulfate; lamivudine
abacavir sulfate; lamivudine; zidovudine
abacavir sulfate;lamivudine
abacavir; lamivudine
abacavir;lamivudine
abaloparatide
abarelix
abatacept
abciximab
abelcet
abemaciclib
abilify
abilify maintena kit
abilify mycite kit
abiraterone acetate
abitrexate
ablavar
ablysinol
abobotulinumtoxina
abraxane
abreva
abrilada
absorica
absorica ld
abstral
acalabrutinib
acamprosate calcium
acanya
acarbose
accolate
accretropin
accrufer
accuneb
accupril
accurbron
accuretic
accutane
acebutolol hydrochloride


In [7]:
# names with aliases and no parentheticals are easy
# e.g abacavir;lamivudine (just split those)
# names with parentheticals are harder - what do they look like
for dn in drug_names:
    if '(' in dn:
        print(dn)

actonel with calcium (copackaged)
aminosyn 10% (ph6)
aminosyn 7% (ph6)
aminosyn 8.5% (ph6)
antara (micronized)
atropine (autoinjector)
clobetasol propionate (emollient)
cytoxan (lyophilized)
ddavp (needs no refrigeration)
desmopressin acetate (needs no refrigeration)
dextrose 5%, sodium chloride 0.2% and potassium chloride 15meq (k)
dextrose 5%, sodium chloride 0.2% and potassium chloride 20meq (k)
dextrose 5%, sodium chloride 0.2% and potassium chloride 5meq (k)
dextrose 5%, sodium chloride 0.45% and potassium chloride 20meq (k) in plastic container
dicyclomine hydrochloride (preservative free)
doxil (liposomal)
doxorubicin hydrochloride (liposomal)
enoxaparin sodium (preservative free)
epinephrine (autoinjector)
epinephrine (copackaged)
evzio (autoinjector)
excedrin (migraine)
famotidine preservative free (pharmacy bulk)
fenofibrate (micronized)
ferric hexacyanoferrate(ii)
glyburide (micronized)
hc (hydrocortisone)
hydroserpine plus (r-h-h)
infuvite pediatric (pharmacy bulk package)


In [5]:
# thankfully there aren't many, and most aren't drug or alias names
# maybe the relevant drugs are already present and we can just drop ()s
drug_name_set = set(drug_names)
names_to_check = ['premarin', 'cycrin', 'hydrocortisone', 'ampicillin',
                 'sulfabenzamide', 'sulfacetamide', 'sulfathiazole',
                 'sulfadiazine', 'sulfamerazine', 'sulfamethazine']
for ntc in names_to_check:
    print('{0}: {1}'.format(ntc, ntc in drug_name_set))

premarin: True
cycrin: True
hydrocortisone: True
ampicillin: False
sulfabenzamide: False
sulfacetamide: False
sulfathiazole: False
sulfadiazine: True
sulfamerazine: False
sulfamethazine: False


In [8]:
# okay, to process these into individual drugs, let's do this
# 1. drop all text after the first occurrence of a '('
# 2. split remaining text by ';'
# 3. manually add our list of drugs from ()s [names_to_check above]
final_drug_names = set()
for dn in drug_names:
    dn = dn.strip()
    # drop all text following a paren
    pindex = dn.find('(')
    if pindex >= 0:
        dn = dn[:pindex]
    # and split by ';' for multiple names
    dn = dn.split(';')
    for n in dn:
        final_drug_names.add(n.strip())
# and add all of our manually gathered names
for n in names_to_check:
    final_drug_names.add(n)
# how'd we do?
final_drug_names = sorted(final_drug_names)
print('\n'.join(final_drug_names[:50]))

8-hour bayer
8-mop
a-hydrocort
a-methapred
a-poxide
a.p.l.
a/t/s
abacavir
abacavir and lamivudine
abacavir sulfate
abacavir sulfate and lamivudine
abacavir sulfate, lamivudine and zidovudine
abaloparatide
abarelix
abatacept
abciximab
abelcet
abemaciclib
abilify
abilify maintena kit
abilify mycite kit
abiraterone acetate
abitrexate
ablavar
ablysinol
abobotulinumtoxina
abraxane
abreva
abrilada
absorica
absorica ld
abstral
acalabrutinib
acamprosate calcium
acanya
acarbose
accolate
accretropin
accrufer
accuneb
accupril
accurbron
accuretic
accutane
acebutolol hydrochloride
aceon
acephen
acetadote
acetaminophen
acetaminophen and codeine phosphate


In [9]:
# fine, let's dump those for use in our word vector stuff
with open('fda_approved/fda_approved.processed.names', 'w') as outfile:
    outfile.write('\n'.join(final_drug_names))
    outfile.write('\n')